In [ ]:
import cv2
import numpy as np

# tracking features. Identifying corners through shi-tomasi corner detector method

def find_features(gray_images):
    points = cv2.goodFeaturesToTrack(gray_images, mask=None, maxCorners=100, qualityLevel=0.1, minDistance=10, blockSize=5)
    
    return points

#Lucas Kanade sparse optical flow model
def LK_model(gray_t_1,gray_t, prev_points):
    points,status,error = cv2.calcOpticalFlowPyrLK(gray_t_1,gray_t,prev_points,None,winSize=(10,10),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))
    
    return (points,status,error)

#driver script
capture = cv2.VideoCapture("Highway Traffic USA.mp4")
lk_params = dict(winSize=(15,15),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))
color = np.random.randint(0,255,(100,3))
ret,prev_frame = capture.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_points = find_features(prev_gray)
mask_image = np.zeros_like(prev_frame)

#reading further frames
while(1):
    ret, frame = capture.read()
    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    current_points, status, error = LK_model(prev_gray,current_gray,prev_points)
    
    #extracting successful points
    if current_points is not None:
        current_points = current_points[status == 1]
        old_points = prev_points[status == 1]
    
    #performing mapping
    for i, (current, prev) in enumerate(zip(current_points,old_points)):
        a,b = current.ravel()
        c,d = prev.ravel()
        #creating black screen where lines are drawn from a-d to track the flow of the pixels
        mask_image = cv2.line(mask_image,(int(a),int(b)),(int(c),int(d)),color[i].tolist(),2)
        
        # plotting the features detected
        frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)
        
        img = cv2.add(frame,mask_image)
        #image = frame
        
        #display image
        cv2.imshow('frame',img)
        #cv2.imshow('mask image',mask_image)
        #cv2.imshow('mask image',image)
        k = cv2.waitKey(5)
        
        if k == 27:
            cv2.destroyAllWindows()
            break
            
        #set the current frame as the t-1 frame and update the current points at the prvious points (t-1). Basically updating the frame
        prev_gray = current_gray.copy()
        prev_points = current_points.reshape(-1,1,2)

